In [4]:

%matplotlib inline
import requests
import re
import time
import os
import urllib.request
from typing import Union
import csv
import statistics
import pdf2docx
import csv
import pickle

import camelot
import typing
import io
import numpy as np
from docx import Document
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
%config Completer.use_jedi = False
from pathlib import Path
# import tqdm.notebook.tqdm as tqdm
from tqdm import tqdm_notebook
%matplotlib widget
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format=u'%(filename)+13s [ LINE:%(lineno)-4s] %(levelname)-8s %(message)s',
                    level=logging.DEBUG)

logger = logging.getLogger(__name__)

from docx2python import docx2python

import xlrd


In [22]:
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\83352_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\83352_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xls"

file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\86951_2017_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xls"
folder = Path(r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls')

for f in os.listdir(folder):
    try:
        df = pd.read_excel(folder / file, engine='xlrd')
        print(df.columns)
    except Exception as ex:

        print('err',ex)

In [8]:
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\178479_2020_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xls"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\86950_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xls"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\84833_2017_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xls"
weird_file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\84720_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii_(ne_ukazany_FIO).xls"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\84719_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii_(ne_ukazany_FIO).xls"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\83297_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\83314_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii_(FBSEU).xlsx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\83337_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\83345_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\83352_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xls"
# мб если нет учреждения и осталась одна не null колонка - назначать ее учреждением? + 2
# так же как и в пдф - если нет учреждения и общее значение на две колонки - значит это учреждение 
# или добавлять в учреждение заголовок таблицы?
#  TODO: в дополнение к стандартному дропу сделать дроп колонок где нет значений длинее 3
# df = pd.read_excel(file, engine='xlrd', ignore_workbook_corruption=True)

# wb = xlrd.open_workbook(file, ignore_workbook_corruption=True)
# df = pd.read_excel(wb, header=None)
# df
# df = pd.read_excel(file)
# df
# df.dropna(axis=1, how='all').fillna(method='ffill', axis=1).head(20)#['Unnamed: 12']

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Сведения о среднемесячной заработной плате рук...,NaN,NaN
1,NaN,NaN,NaN
2,Должность,ФИО,"Среднемесячная заработная плата, руб."
3,Директор,Кутилин Е.А.,198368.7
4,Заместитель директора,Бида П.А.,132569
5,Заместитель директора,Голованов И.С.,158406.1
6,Заместитель директора,Дракин В.В.,37410
7,Заместитель директора,Матвеенко А.А.,236947
8,Заместитель директора,Родионова О.А.,134787
9,Заместитель директора,Скопцов Ю.В.,144796


In [15]:
class ExcelParser:

    @staticmethod
    def find_ok_cols(cols: tuple) -> dict[int, str]:
        '''ищем заголовки в таблице'''
        
        cols = list(map(str, cols))
        cols = list(map(str.lower, cols))
        ok_cols = 0
        result = {} # 'n_of_col':'new_name'

        for i, col in enumerate(cols):
            if len(result) > 3 or i > 20:
                return result

            if re.search(pattern='(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество)', string=col):
                result[i] = 'name'
                ok_cols+=1

            elif re.search(pattern='(рублей|руб|cреднемесячная|зарпл.|плат[ы, а]|заработн[ой, ая] плат[а, ы]|cреднемесячн[ая, ой]|зарплат[а, ной, ы])', string=col):
                result[i] = "salary"
                ok_cols+=1
                
            elif re.search(pattern='(должност[ь, и, ей])', string=col): 
                result[i] = 'position'
                ok_cols+=1
                
            elif re.search(pattern='(предприяти[е,я]|учреждени[е,я]|юридическое лицо)', string=col):
                result[i] = 'department'
                ok_cols+=1

        return result

    def rename_cols(self, df: pd.DataFrame, new_cols: dict[int, str]):
        for k,v in new_cols.items():
            df.rename(columns={ df.columns[k]: v }, inplace = True)
        return df 
    
    def find_department(df: pd.DataFrame) -> pd.DataFrame:
        pass 

    def detect_headers(self, df: pd.DataFrame)-> pd.DataFrame:
        for row in df.itertuples():
            index = row[0]
            new_cols = self.find_ok_cols(row[1:])
            if new_cols:
                if 'department' not in new_cols.keys():
                    df = self.find_department(df)
                    
                    
                else
                    df = self.rename_cols(df, new_cols)
                    return df

        # 1. надо найти заголовки
        # 2. если нет учреждения - искать в сплитах
        # for _, row in df.itertuples():
        #     pass


    def parse_file(self, filename: Path):
        df = pd.read_excel(filename, header=None)
        #TODO: отлов ошибок при чтении разных форматов
        df = df.dropna(axis=1, how='all').fillna(method='ffill', axis=1)                
        df = self.detect_headers(df)
        return df



excel_parser = ExcelParser()
excel_parser.parse_file(file)
#new_cols = excel_parser.detect_headers(df)
#df.rename(columns={ df.columns[1]: "your value" }, inplace = True)
# temp_df=  df.copy()


#temp_df.dropna(axis=1, how='all').fillna(method='ffill', axis=1)        
# temp_df 


IndentationError: expected an indented block (Temp/ipykernel_17904/3781532858.py, line 58)

In [31]:
df = df.dropna(axis=1, how='all').fillna(method='ffill', axis=1)                

for row in df.itertuples():
    row = list(row)[1:]
    print(row[0] == None)

    продолжить писать правила для нахождения департамента

    # print(len(set(row))<2)
    print('=====')

False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====
False
=====


In [70]:
# df.rename(columns={ df.columns[1]: "your value" }, inplace = True)


,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Сведения о среднемесячной заработной плате рук...,NaN,NaN
1,NaN,NaN,NaN
2,Должность,ФИО,"Среднемесячная заработная плата, руб."
3,Директор,Кутилин Е.А.,198368.7
4,Заместитель директора,Бида П.А.,132569
5,Заместитель директора,Голованов И.С.,158406.1
6,Заместитель директора,Дракин В.В.,37410
7,Заместитель директора,Матвеенко А.А.,236947
8,Заместитель директора,Родионова О.А.,134787
9,Заместитель директора,Скопцов Ю.В.,144796
